# Import libraries

In [1]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import data

In [3]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
full_data = pd.read_csv(repo_url + 'full_data.csv')
distance_matrix = pd.read_csv(repo_url + 'distance_matrix.csv',header=None)
vehicle_matrix = pd.read_excel(repo_url + 'vhc_matrix_city_excluded.xlsx')

In [4]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,238
0,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
1,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
2,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
3,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
4,4.1759,5.6066,2.3205,5.8087,inf,3.2004,8.0162,8.1495,8.3665,2.7008,...,67.9534,68.5994,68.6829,69.3463,69.6334,72.1121,72.3058,74.0855,77.0580,89.0782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,73.6675,...,112.4984,24.8423,28.5708,10.1314,115.5113,inf,10.1570,31.4998,18.1614,26.9294
235,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,112.6921,34.1475,19.4076,30.5988,115.7050,10.1570,inf,22.3366,35.7624,17.7661
236,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,75.6409,...,82.9661,55.4904,8.6894,51.9416,85.9790,31.4998,22.3366,inf,57.1422,27.3442
237,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,78.6133,...,117.4443,20.1333,54.1762,9.3360,120.4571,18.1614,35.7624,57.1422,inf,52.5393


In [5]:
distance_matrix.replace(np.inf, 0, inplace=True)

In [6]:
full_data.fillna(0, inplace=True)

In [7]:
demands_wed =  [0] + full_data['wed_tickets'].astype(int).values.tolist()
demands_thu =  [0] + full_data['thu_tickets'].astype(int).values.tolist()
demands_fri =  [0] + full_data['fri_tickets'].astype(int).values.tolist()
demands_sat =  [0] + full_data['sat_tickets'].astype(int).values.tolist()

In [8]:
venoge = [0]
venoge.extend(full_data['distance_to_venoge_km'].values.tolist())
df_column = pd.DataFrame(venoge)
df_row = pd.DataFrame(full_data['distance_to_venoge_km'].values.tolist()).T

In [9]:
distance_matrix = pd.concat([df_row,distance_matrix],ignore_index=True)
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,238
0,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,5.0800,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,0.0000,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.6786,0.0000,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,2.8743,3.9804,0.0000,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,4.0302,3.0074,4.1825,0.0000,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,73.6675,...,112.4984,24.8423,28.5708,10.1314,115.5113,0.0000,10.1570,31.4998,18.1614,26.9294
236,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,112.6921,34.1475,19.4076,30.5988,115.7050,10.1570,0.0000,22.3366,35.7624,17.7661
237,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,75.6409,...,82.9661,55.4904,8.6894,51.9416,85.9790,31.4998,22.3366,0.0000,57.1422,27.3442
238,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,78.6133,...,117.4443,20.1333,54.1762,9.3360,120.4571,18.1614,35.7624,57.1422,0.0000,52.5393


In [10]:
distance_matrix.columns = range(1, len(distance_matrix.columns) + 1)
distance_matrix

,1,2,3,4,5,6,7,8,9,10,...,230,231,232,233,234,235,236,237,238,239
0,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,5.0800,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,0.0000,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.6786,0.0000,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,2.8743,3.9804,0.0000,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,4.0302,3.0074,4.1825,0.0000,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,73.6675,...,112.4984,24.8423,28.5708,10.1314,115.5113,0.0000,10.1570,31.4998,18.1614,26.9294
236,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,112.6921,34.1475,19.4076,30.5988,115.7050,10.1570,0.0000,22.3366,35.7624,17.7661
237,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,75.6409,...,82.9661,55.4904,8.6894,51.9416,85.9790,31.4998,22.3366,0.0000,57.1422,27.3442
238,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,78.6133,...,117.4443,20.1333,54.1762,9.3360,120.4571,18.1614,35.7624,57.1422,0.0000,52.5393


In [11]:
distance_matrix.insert(0,0,df_column)
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0.00,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,0.74,0.0000,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.72,2.6786,0.0000,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,3.51,2.8743,3.9804,0.0000,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,3.98,4.0302,3.0074,4.1825,0.0000,5.8087,7.6315,6.1501,3.6034,7.4234,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.05,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,...,112.4984,24.8423,28.5708,10.1314,115.5113,0.0000,10.1570,31.4998,18.1614,26.9294
236,72.24,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,...,112.6921,34.1475,19.4076,30.5988,115.7050,10.1570,0.0000,22.3366,35.7624,17.7661
237,74.02,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,...,82.9661,55.4904,8.6894,51.9416,85.9790,31.4998,22.3366,0.0000,57.1422,27.3442
238,77.00,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,...,117.4443,20.1333,54.1762,9.3360,120.4571,18.1614,35.7624,57.1422,0.0000,52.5393


In [12]:
distance_matrix = distance_matrix.round().astype(int)
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0,1,3,4,4,4,4,5,5,5,...,68,69,69,69,70,72,72,74,77,89
1,1,0,3,3,4,4,4,5,5,4,...,68,69,69,69,70,72,72,74,77,89
2,3,3,0,4,3,6,6,4,3,4,...,66,71,71,71,68,74,74,76,79,91
3,4,3,4,0,4,2,6,7,6,8,...,68,69,69,69,70,72,72,74,77,89
4,4,4,3,4,0,6,8,6,4,7,...,64,71,71,72,66,74,75,76,79,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72,72,74,72,74,72,72,76,76,76,...,112,25,29,10,116,0,10,31,18,27
236,72,72,74,72,75,72,72,76,76,76,...,113,34,19,31,116,10,0,22,36,18
237,74,74,76,74,76,74,74,78,78,78,...,83,55,9,52,86,31,22,0,57,27
238,77,77,79,77,79,77,76,81,81,81,...,117,20,54,9,120,18,36,57,0,53


In [51]:
subset_distance_matrix=distance_matrix.values.tolist()
len(subset_distance_matrix)

240

In [54]:
subset_demands = demands_wed
len(subset_demands)

240

In [61]:
sum(subset_demands)

14814

In [15]:
#distance_matrix = distance_matrix.values.tolist()

In [66]:
real_capacities=vehicle_matrix['Places'].values.tolist()

In [67]:
real_capacities

[16, 91, 50, 20, 127, 100]

In [17]:
capacities=[9000,600] + [300]*13 + [100]*16 + [50]*5

In [70]:
capacities =[9000] + [600] + [300]*4 + [200]*9 + real_capacities*35# [9000,500,500]

In [69]:
capacities

[9000,
 600,
 300,
 300,
 300,
 300,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100,
 16,
 91,
 50,
 20,
 127,
 100]

In [60]:
sorted(demands_wed,reverse=True)

[8625,
 529,
 271,
 246,
 237,
 223,
 172,
 165,
 148,
 147,
 137,
 133,
 119,
 108,
 103,
 98,
 83,
 78,
 77,
 74,
 74,
 73,
 71,
 67,
 65,
 59,
 56,
 52,
 51,
 51,
 50,
 48,
 48,
 44,
 42,
 42,
 41,
 40,
 40,
 39,
 38,
 38,
 38,
 37,
 37,
 37,
 37,
 36,
 32,
 31,
 31,
 30,
 29,
 28,
 27,
 27,
 27,
 26,
 26,
 26,
 23,
 23,
 21,
 21,
 21,
 20,
 20,
 19,
 19,
 19,
 18,
 18,
 17,
 17,
 17,
 17,
 17,
 17,
 16,
 16,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,

In [73]:
distance_matrix_list = distance_matrix.values.tolist()

# Draft model

In [74]:


from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(subset_distance_matrix, subset_demands, capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = subset_distance_matrix
    data['demands'] = subset_demands
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = len(capacities)
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console, listing unused vehicles separately."""
    total_distance = 0
    total_load = 0
    unused_vehicles = []

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        capacity = data['vehicle_capacities'][vehicle_id]
        route_output = f"Route for vehicle {vehicle_id} (Capacity: {capacity}):\n"

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            route_load += demand
            route_output += (
                f" {node_index} ->"# (Demand: {demand}, Current Load: {route_load}) ->"
            )
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        end_node = manager.IndexToNode(index)
        route_output += f" {end_node} (Load: {route_load})\n"

        if route_load == 0:
            unused_vehicles.append(vehicle_id)
            continue  # Skip printing empty routes

        route_output += f"Distance of the route: {route_distance / 10:.2f} km\n"
        route_output += f"Load of the route: {route_load}\n"
        print(route_output)

        total_distance += route_distance
        total_load += route_load

    print(f"Total distance of all routes: {total_distance / 10:.2f} km")
    print(f"Total load of all routes: {total_load}")
    
    if unused_vehicles:
        print(f"Unused vehicles: {unused_vehicles}")
        print(f"Number of unused vehicles: {len(unused_vehicles)}")


def main():
    """Solve the CVRP problem."""

    ## TODO: Complete here
    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix=distance_matrix_list, subset_demands=demands_wed, capacities=capacities)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


main()

Route for vehicle 0 (Capacity: 9000):
 0 -> 163 -> 221 -> 201 -> 203 -> 209 -> 238 -> 231 -> 124 -> 123 -> 91 -> 39 -> 7 -> 0 (Load: 8999)
Distance of the route: 21.00 km
Load of the route: 8999

Route for vehicle 1 (Capacity: 600):
 0 -> 22 -> 12 -> 14 -> 0 (Load: 598)
Distance of the route: 3.20 km
Load of the route: 598

Route for vehicle 2 (Capacity: 300):
 0 -> 35 -> 61 -> 155 -> 175 -> 161 -> 0 (Load: 299)
Distance of the route: 9.10 km
Load of the route: 299

Route for vehicle 3 (Capacity: 300):
 0 -> 13 -> 53 -> 59 -> 76 -> 113 -> 133 -> 106 -> 15 -> 0 (Load: 298)
Distance of the route: 7.20 km
Load of the route: 298

Route for vehicle 4 (Capacity: 300):
 0 -> 51 -> 79 -> 90 -> 23 -> 2 -> 0 (Load: 300)
Distance of the route: 5.50 km
Load of the route: 300

Route for vehicle 5 (Capacity: 300):
 0 -> 3 -> 1 -> 0 (Load: 283)
Distance of the route: 0.80 km
Load of the route: 283

Route for vehicle 6 (Capacity: 200):
 0 -> 135 -> 205 -> 179 -> 176 -> 160 -> 136 -> 150 -> 67 -> 86 ->

In [59]:
for index,demand in enumerate(full_data['thu_tickets'].values.tolist()):
    if demand == 0:
        print(index)
    

31
51
53
70
71
92
100
102
152
158
169
179
181
191
208
211
213
214
217
219
223
225
228
229
234
236
